<a href="https://colab.research.google.com/github/kundana22bce8985/knowledge-based-search-engine/blob/main/your_notebook_fixed_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers faiss-cpu PyPDF2 ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.1 MB/s eta 0:00:00


In [2]:
import PyPDF2
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from IPython.display import display
import ipywidgets as widgets


In [3]:
# Embedding model (MiniLM)
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# LLM model (FLAN-T5)
llm_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(llm_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(llm_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + " "
    return text

pdf_text = extract_text_from_pdf("Resume.pdf")

In [6]:
# Chunk the extracted text
def chunk_text(text, chunk_size=500, chunk_overlap=50):
    chunks = []
    for i in range(0, len(text), chunk_size - chunk_overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

chunks = chunk_text(pdf_text)

In [7]:
# Create embeddings for each chunk
chunk_embeddings = embed_model.encode(chunks, convert_to_numpy=True)

# Build FAISS index
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)


In [8]:
def retrieve_relevant_chunks(question, top_k=3):
    q_embedding = embed_model.encode([question], convert_to_numpy=True)
    distances, indices = index.search(q_embedding, top_k)
    return " ".join([chunks[i] for i in indices[0]])

def answer_question_rag(question, top_k=3):
    context = retrieve_relevant_chunks(question, top_k)
    prompt = f"Answer the question based on the context:\nContext: {context}\nQuestion: {question}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    outputs = llm_model.generate(**inputs, max_length=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [9]:
question_box = widgets.Text(
    value='',
    placeholder='Type your question here',
    description='Question:',
    disabled=False
)
output_box = widgets.Output()

def on_button_click(b):
    with output_box:
        output_box.clear_output()
        question = question_box.value
        answer = answer_question_rag(question)
        print("Answer:", answer)

button = widgets.Button(description="Ask")
button.on_click(on_button_click)

display(question_box, button, output_box)


Text(value='', description='Question:', placeholder='Type your question here')

Button(description='Ask', style=ButtonStyle())

Output()

Answer: Programming Languages Java, Python Web Development HTML, CSS Machine Learning Algorithms, Datasets Platforms VS Code, Eclipse Projects Plant Disease Detection •Developed a deep learning model using TensorFlow and Keras to classify plant leaves as healthy or affected by diseases. •Collected and cleaned image data, applying data augmentation techniques to improve robustness. •Experimented with Convolutional Neural Netwo ns at school level.
